# CTFishPy Ingest Tutorial

This notebook will walk you through how to ingest a multi-fish tiff scan and seperate them into separate dicom files. Make sure to also sort out your metadata.

Please make sure you have ctfishpy installed.


Before we start let's import ctfishpy and required libraries then initialise the ctreader and lumpfish:


In [1]:
#Import ctfishpy and initialise ctreader
from pathlib2 import Path
import pandas as pd
import numpy as np
import ctfishpy
import napari

ctreader = ctfishpy.CTreader(data_path=None)
lump = ctfishpy.Lumpfish()

## Step 1 - define paths, names and read scan

Define the path to your tif folder

Change the detection scale to 40 to make the interface quicker

Set the slice range to read first for instance (900,1100) will read 200 slices in the middle of the scan, 
When you are ready to read the whole scan just set slice_range to `None`

Number your fish by editing `fish_nums`

This step will take the longest (10-15 mins) but you only have to do it once

In [2]:
# tif_path = Path("path/to/005-009_tifs") # Make sure this is the tif file and not the projections
tif_path = Path("/home/ak18001/Data/HDD/uCT/MISC/QIAO_dirty/QT_005-009 [2021-06-03 16.38.19]/QT_005-009_[tifs]") # Make sure this is the tif file and not the projections

# output_path = Path("path/to/output/")
output_path = Path("/home/ak18001/Data/HDD/uCT/MISC/QIAO_dirty/")

original_scale = 100 # full scale of scan
detection_scale = 40 # detection scale
slice_range = None # slice range, can be a tuple eg (500,600) or None to read the whole scan

dataset_name = 'EK' # eg EK, AK, QT
fish_nums = [5,6,7,8,9] # numbers of fish
voxel_size_x = 0.0020
voxel_size_y = 0.0020
voxel_size_z = 0.0020

ct = lump.read_tiff(tif_path, r=slice_range, scale=original_scale)

[CTFishPy] Reading uCT scan: /home/ak18001/Data/HDD/uCT/MISC/QIAO_dirty/QT_005-009 [2021-06-03 16.38.19]/QT_005-009_[tifs]


100%|██████████| 1946/1946 [10:00<00:00,  3.24it/s]


(1946, 2000, 1774)


## Step 2

Use lumpfish to detect tubes at a 40% scale to make the interface faster

This will start a window with all the detection parameters for you to tune

In [3]:
scale_40 = lump.rescale(ct, detection_scale)
# detect tubes
viewer = napari.Viewer(show=False)
circle_dict = lump.detectTubes(viewer, scale_40)

## Step 3

Label the order of the fish - this is usually marked by a sticker or a cap on the tube of the first fish, then go clockwise. If there is a fish in the middle this is the last one.

In [4]:
# label order
viewer = napari.Viewer(show=False)
ordered = lump.labelOrder(viewer, circle_dict)

## Step 4

choose the output folder to save your scans as .dicoms, this will label them according to `fish_nums` which you set earlier.

Finally crop the fish according to the detection and the order, 
then align each fish so that the dorsal fin is pointing upwards.

This will also create a temporary metadata file for you to fill out.

#TODO make projections

In [5]:
cropped_cts = lump.crop(ct, ordered, scale=[detection_scale,original_scale])

df = pd.DataFrame(columns = ['ak_n', 'Dataset', 'old_n', 'age', 'age(old)', 'genotype', 'strain',
       'name', 'shape', 'size', 'VoxelSizeX', 'VoxelSizeY', 'VoxelSizeZ',
       're-uCT scan', 'Comments', 'Phantom', 'Scaling Value', 'Arb Value',
       'angle', 'center'], 
                   index = fish_nums)

for i,cropped in enumerate(cropped_cts):
    num = fish_nums[i]

    spin_viewer = napari.Viewer(show=False)
    angle, center = lump.spin(spin_viewer, cropped)

    #TODO make dataframe of new metadata

    final_ct = ctreader.rotate_array(cropped, angle, is_label=False, center=center)
    ctreader.write_dicom(output_path/f"{num}.dcm", final_ct)

    print('num and shape', num, cropped.shape)
    print('angle and center', angle, center)

    df.loc[num]['Dataset']      = dataset_name
    df.loc[num]['shape']        = final_ct.shape
    df.loc[num]['size']         = final_ct.size
    df.loc[num]['VoxelSizeX']   = voxel_size_x
    df.loc[num]['VoxelSizeY']   = voxel_size_y
    df.loc[num]['VoxelSizeZ']   = voxel_size_z
    df.loc[num]['angle']        = angle
    df.loc[num]['center']       = center

df.to_csv(output_path / "temp_metadata.csv")

num and shape 5 (1946, 560, 560)
angle and center 199 None
num and shape 6 (1946, 544, 544)
angle and center 276 None
num and shape 7 (1946, 594, 594)
angle and center 3 None
num and shape 8 (1946, 584, 584)
angle and center 89 None
num and shape 9 (1946, 584, 584)
angle and center 340 None
